<a href="https://colab.research.google.com/github/ailuj/BayesianDeepLearning/blob/playground/MC%20Dropout%20Regression%20Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"batuhanipekci","key":"8d50f8f4e095ae646c2912649292ed60"}'}

In [0]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d wendykan/lending-club-loan-data 


 97% 233M/240M [00:07<00:00, 34.8MB/s]
100% 240M/240M [00:07<00:00, 34.5MB/s]


In [0]:
from zipfile import ZipFile
zip_file = ZipFile('lending-club-loan-data.zip')
zip_file.namelist()
loan = pd.read_csv(zip_file.open('loan.csv'))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
loan = loan[(loan['loan_status'] != 'Current')]
selected = loan[['total_pymnt', 'annual_inc', 'int_rate','loan_amnt','purpose', 'loan_status']]
selected = selected.dropna()
target = selected.apply(lambda x: (x['total_pymnt'] - x['loan_amnt'])/x['loan_amnt'], axis = 1)
data = selected.drop(columns = ['total_pymnt', 'loan_amnt', 'loan_status'])
data = pd.get_dummies(data)
data.head()

,annual_inc,int_rate,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,24000.0,10.65,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,30000.0,15.27,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,12252.0,15.96,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,49200.0,13.49,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,36000.0,7.90,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

scaler_tr = preprocessing.MinMaxScaler().fit(X_train)
X_train = scaler_tr.transform(X_train)

scaler_test = preprocessing.MinMaxScaler().fit(X_test)
X_test = scaler_test.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# MLP Architecture


In [0]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
import time

In [0]:
dropout_prob = 0.05 # dropout should be between 0.05 and 0.5
n_epochs = 100
tau = 1.0

n_hidden = [100,100] # 2 hidden layers, each with 100 nodes
N = X_train.shape[0]
input_dim = X_train.shape[1]
output_dim = 1

batch_size = 5000


In [0]:
y_train.shape


(191349,)

In [0]:
from keras import Model, Input
from keras.layers import Dropout
from keras.layers import Dense

def mlp(input_dim, n_hidden, dropout_prob, train_activation, MCDropout = False):
  inputs = Input(shape=(X_train.shape[1],))
  inter = Dropout(dropout_prob)(inputs, training=True)
  inter = Dense(n_hidden[0], activation=train_activation)(inter)
  for i in range(len(n_hidden) - 1):
    inter = Dropout(dropout_prob)(inter, training=True)
    inter = Dense(n_hidden[i+1], activation=train_activation)(inter)
  if MCDropout == True:
    inter = Dropout(dropout_prob)(inter, training=True)
  outputs = Dense(1)(inter) # Not that we don't put activation here
  model = Model(inputs, outputs)
  
  return model

In [0]:
from keras.layers.advanced_activations import LeakyReLU

model = mlp(input_dim = input_dim, n_hidden = n_hidden, 
            dropout_prob = dropout_prob, train_activation = 'relu',MCDropout = False)
#model.compile(loss = 'mean_squared_error',optimizer='adam')
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epochs, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/100
191349/191349 [==============================] - 1s 7us/step - loss: 0.1378 - mean_absolute_error: 0.2927
Epoch 2/100
191349/191349 [==============================] - 1s 5us/step - loss: 0.1365 - mean_absolute_error: 0.2902
Epoch 3/100
191349/191349 [==============================] - 1s 5us/step - loss: 0.1366 - mean_absolute_error: 0.2905
Epoch 4/100
191349/191349 [==============================] - 1s 5us/step - loss: 0.1362 - mean_absolute_error: 0.2897
Epoch 5/100
191349/191349 [==============================] - 1s 5us/step - loss: 0.1362 - mean_absolute_error: 0.2904
Epoch 6/100
191349/191349 [==============================] - 1s 5us/step - loss: 0.1361 - mean_absolute_error: 0.2897
Epoch 7/100
191349/191349 [==============================] - 1s 5us/step - loss: 0.1360 - mean_absolute_error: 0.2901
Epoch 8/100
191349/191349 [==============================] - 1s 5us/step - loss: 0.1360 - mean_absolute_error: 0.2899
Epoch 9/100
191349/191349 [=============================

In [0]:
pred = model.predict(X_test, verbose=1)

94247/94247 [==============================] - 2s 17us/step


In [0]:
from sklearn.metrics import mean_squared_error
import numpy as np
mean_squared_error(pred, y_test)

0.1354552632402013

# Bayesian MLP with Dropout Trick

In [0]:
model = mlp(input_dim = input_dim, n_hidden = n_hidden, 
            dropout_prob = dropout_prob, train_activation = 'relu', 
            MCDropout = True)
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epochs, verbose=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


In [0]:
T = 1000
        
probs = []
for _ in range(T):
    probs += [model.predict(X_test,verbose=1)]

94247/94247 [==============================] - 3s 31us/step


In [0]:
import numpy as np
predictive_mean = np.mean(probs, axis=0)
predictive_variance = np.var(probs, axis=0)
                                  
l = 100 
# I need to check out how to choose l and tau

tau = l**2 * (1 - dropout_prob) / (2 * N * 1)
predictive_variance += tau**-1


0.1354552632402013

In [0]:
# Accuracy
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,predictive_mean)


0.1353316897423303

In [0]:
probs

[array([[-0.06503227],
        [-0.04200393],
        [-0.04073668],
        ...,
        [-0.04207636],
        [-0.10657059],
        [-0.03526378]], dtype=float32), array([[-0.07117879],
        [-0.03691291],
        [-0.04013819],
        ...,
        [-0.06498929],
        [-0.08967198],
        [-0.03599197]], dtype=float32), array([[-0.0565348 ],
        [-0.04727753],
        [-0.04300864],
        ...,
        [-0.04244656],
        [-0.14277047],
        [-0.0393939 ]], dtype=float32), array([[-0.05014672],
        [-0.03874834],
        [-0.04583411],
        ...,
        [-0.04428134],
        [-0.10389436],
        [-0.03270315]], dtype=float32), array([[-0.05942667],
        [-0.03932481],
        [-0.04101357],
        ...,
        [-0.0433066 ],
        [-0.10513546],
        [-0.04069372]], dtype=float32), array([[-0.05166199],
        [-0.04079667],
        [-0.04573719],
        ...,
        [-0.05203283],
        [-0.11134376],
        [-0.03820446]], dtype=float32